# Binary Diagnostic

The analysis that follows pertains to the third day of the [Python Problem-Solving Bootcamp](https://mathspp.com/pythonbootcamp).

In the analysis that follows you may be confronted with code that you do not understand, especially as you reach the end of the explanation of each part.

If you find functions that you didn't know before, remember to [check the docs](https://docs.python.org/3/) for those functions and play around with them in the REPL.
This is written to be increasing in difficulty (within each part of the problem), so it is understandable if it gets harder as you keep reading.
That's perfectly fine, you don't have to understand everything _right now_, especially because I can't know for sure what _your level_ is.

## Part 1 problem statement

(Adapted from [Advent of Code 2021, day 3](https://adventofcode.com/2021/day/3))

You are given a list of binary numbers.
You need to use the binary numbers in that list to generate two new binary numbers (called the **gamma rate** and the **epsilon rate**).

Each bit in the gamma rate can be determined by finding the most common bit in the corresponding position of all numbers in the list.
For example, given the following list:

```txt
00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010
```

Considering only the first bit of each number, there are five `0` bits and seven `1` bits. Since the most common bit is `1`, the first bit of the gamma rate is `1`.

The most common second bit of the numbers in the diagnostic report is `0`, so the second bit of the gamma rate is `0`.

The most common value of the third, fourth, and fifth bits are `1`, `1`, and `0`, respectively, and so the final three bits of the gamma rate are `110`.

So, the gamma rate is the binary number `10110`, or `22` **in decimal**.

The epsilon rate is calculated in a similar way; rather than use the most common bit, the least common bit from each position is used. So, the epsilon rate is `01001`, or `9` **in decimal**. Multiplying the gamma rate (`22`) by the epsilon rate (`9`) gives `198`.

**Use the binary numbers in your input list to calculate the gamma rate and epsilon rate, then multiply them together.** What do you get? (Be sure to represent your answer in decimal, not binary.)

_Using the input file `03_binary_diagnostic.txt`, the result should be 749376._

In [1]:
# IMPORTANT: Set this to the correct path for you!
INPUT_FILE = "03_binary_diagnostic.txt"

### Baseline solution

For the baseline solution, what we want to do is read the whole file into memory.
Then, for each of the columns, we go down the lines of the file counting how many zeroes and ones appear in that column.
After we are done counting zeroes and ones for a given column, we update the gamma and epsilon rates:

In [5]:
with open(INPUT_FILE, "r") as f:
    lines = [line.strip() for line in f]

gamma, epsilon = "", ""
for col in range(len(lines[0])):
    # Count zeroes and ones in this column:
    zeroes, ones = 0, 0
    for line in lines:
        if line[col] == "0":
            zeroes += 1
        else:
            ones += 1
    # Update gamma and epsilon based on the most common bit:
    if zeroes > ones:
        gamma += "0"
        epsilon += "1"
    else:
        gamma += "1"
        epsilon += "0"
        
print(int(gamma, 2) * int(epsilon, 2))

749376


Even though this solution doesn't use many advanced techniques, there is one thing that is **super** useful already:
the built-in `int` can parse binary numbers:

In [6]:
int("101", 2)

5

That's something we will be using a lot, in here.

Another thing that is worth pointing out is that, when we read the file, we used `.strip()` to get rid of the newline character `"\n"` that comes in the end of each line when we use `.readlines()` on the file.

Now we want to know how to improve our code.
The first thing I do is wonder: what would I changed if the numbers in the problem changed a bit?
For example, what would I do if the problem wanted us to count digits in hexadecimal, instead of binary?

If we were dealing with hexadecimal digits (numbers `0` through `9` and letters `"a"` through `"f"`), then I wouldn't want to have 16 variables just for counting:

In [7]:
zeroes = ones = twos = threes = fours = fives = sixes = sevens = eights = nines = a = b = c = d = e = f = 0

That because, on top of this, I would need an `if` block with 16 branches!

So, the first thing we want to do is improve the counting mechanism.

## Convenient counting

Thankfully, we can do this with ease, we just need to use a container (something like a list or a dictionary) to hold the counting results.

I tend to prefer dictionaries, because the key-value system makes it very easy to map any kind of value to its count:

In [10]:
with open(INPUT_FILE, "r") as f:
    lines = [line.strip() for line in f]

gamma, epsilon = "", ""
for col in range(len(lines[0])):
    # Count zeroes and ones in this column:
    counting = {"0": 0, "1": 0}
    for line in lines:
        counting[line[col]] += 1
    # Update gamma and epsilon based on the most common bit:
    if counting["0"] > counting["1"]:
        gamma += "0"
        epsilon += "1"
    else:
        gamma += "1"
        epsilon += "0"
        
print(int(gamma, 2) * int(epsilon, 2))

749376


As you can see, this simplified the code a fair bit already.

If you are wondering about the reason why I initialised the dictionary `counting` as `{"0": 0, "1": 0}` instead of `{}`, think about this:
if `"0"` and `"1"` are not existing keys of the dictionary `counting`, the line `counting[line[col]] += 1` wouldn't work.
We would have to write a an `if` statement to cover the first time we add something to the dictionary:

In [11]:
with open(INPUT_FILE, "r") as f:
    lines = [line.strip() for line in f]

gamma, epsilon = "", ""
for col in range(len(lines[0])):
    # Count zeroes and ones in this column:
    counting = {}
    for line in lines:
        if line[col] not in counting:
            counting[line[col]] = 0
        counting[line[col]] += 1
    # Update gamma and epsilon based on the most common bit:
    if counting["0"] > counting["1"]:
        gamma += "0"
        epsilon += "1"
    else:
        gamma += "1"
        epsilon += "0"
        
print(int(gamma, 2) * int(epsilon, 2))

749376


This is a common pattern in programming: you “look before you leap” (LBYL).
In other words, you make sure you _can_ do what you wanted to do.
In this case, you make sure the key exists before accessing that key in the dictionary.

However, Python tends to follow another code style, that says it's “easier to ask forgiveness than permission” (EAFP).
This code style suggests you should `try` to do what you want to do, and just fix the situation if you end up in trouble.

## EAFP versus LBYL

In Python, specifically, this generally means contrasting a preventive `if` with a `try` block.
For our example, something like this:

In [12]:
with open(INPUT_FILE, "r") as f:
    lines = [line.strip() for line in f]

gamma, epsilon = "", ""
for col in range(len(lines[0])):
    # Count zeroes and ones in this column:
    counting = {}
    for line in lines:
        try:
            counting[line[col]] += 1
        except KeyError:
            counting[line[col]] = 1
    # Update gamma and epsilon based on the most common bit:
    if counting["0"] > counting["1"]:
        gamma += "0"
        epsilon += "1"
    else:
        gamma += "1"
        epsilon += "0"
        
print(int(gamma, 2) * int(epsilon, 2))

749376


Using the EAFP approach is often the preferred way in Python, and this comparison was shown here for the sake of completeness.
You can read more about the choice between EAFP and LBYL in [here](https://mathspp.com/blog/pydonts/eafp-and-lbyl-coding-styles).

In our case, we can avoid the debate altogether by initialising the counting dictionary in the appropriate way, like was shown [above](#Convenient-counting).

## Dictionary with default value

This whole discussion about initialising the dictionary with the default values, versus using an `if` statement to ensure we can access the dictionary, versus a `try: ... except: ...` block, shows that in all three approaches we needed to give some default value to the dictionary.

Wouldn't it be great if there was some version of `dict` that assumed a default value?
Well, today is your lucky day, because there is!

`defaultdict`, from the `collections` module, is what we want.
`defaultdict` behaves just like a regular dictionary, except that you give it a “default value factory”: a function that returns the default values we care about.

In our case, we see that the count of a digit we haven't seen before should be `0`, so we just need a function that returns `0` to use with `defaultdict`.
As it turns out, `int` does the job:

In [13]:
int()

0

Now, we can use `collections.defaultdict` to do the job:

In [15]:
from collections import defaultdict
olympic_medals = defaultdict(int)

In [16]:
olympic_medals["Rodrigo"]

0

Notice how, above, the dictionary knows that I have zero olympic medals (the default value for any human being), even though I never told the dictionary explicitly how many medals I have.

We can use a similar thing for our counting:

In [17]:
from collections import defaultdict

with open(INPUT_FILE, "r") as f:
    lines = [line.strip() for line in f]

gamma, epsilon = "", ""
for col in range(len(lines[0])):
    # Count zeroes and ones in this column:
    counting = defaultdict(int)
    for line in lines:
        counting[line[col]] += 1
    # Update gamma and epsilon based on the most common bit:
    if counting["0"] > counting["1"]:
        gamma += "0"
        epsilon += "1"
    else:
        gamma += "1"
        epsilon += "0"
        
print(int(gamma, 2) * int(epsilon, 2))

749376


In our case, because we only had two digits, initialising the dictionary by hand or using `defaultdict` was approximately the same work.
`defaultdict` becomes more useful if we have a lot of different things we might want to count, or if we can't know in advance _what_ things will be counted.

However, Python has an even _better_ way to count things:

## Counter

The `collections` module has another useful tool for us: a `Counter`!
A `Counter` does exactly what it says on the tin: it counts things:

In [20]:
from collections import Counter
mississippi_letters = Counter("Mississippi")
print(mississippi_letters)

Counter({'i': 4, 's': 4, 'p': 2, 'M': 1})


It behaves a lot like a dictionary:

In [22]:
mississippi_letters["i"]

4

But it includes other useful methods, like the `most_common`:

In [23]:
mississippi_letters.most_common(2)

[('i', 4), ('s', 4)]

`Counter`s also have a default value of `0`:

In [25]:
mississippi_letters["z"]

0

Thus, we can replace our `defaultdict` with a `Counter`, _and_ make use of the `most_common` method:

In [28]:
from collections import Counter

with open(INPUT_FILE, "r") as f:
    lines = [line.strip() for line in f]

gamma, epsilon = "", ""
for col in range(len(lines[0])):
    # Count zeroes and ones in this column:
    counting = Counter()
    for line in lines:
        counting[line[col]] += 1
    # Update gamma and epsilon based on the most common bit:
    bit, _ = counting.most_common(1)[0]
    if bit == "0":
        gamma += "0"
        epsilon += "1"
    else:
        gamma += "1"
        epsilon += "0"
        
print(int(gamma, 2) * int(epsilon, 2))

749376


If you are doing any sort of _counting_ in Python, you should automatically think about `Counter`!

## Counting the whole column

What is annoying about this task is that we are asked to go through lines to count things in specific columns.
If the columns were actually rows, counting would be much easier.
In other words, it would be great if we could reformat the input, that looks like

```txt
111
000
101
101
111
```

into something that looks like

```txt
10111
10001
10111
```

If we do so, counting becomes _much_ easier.

As it turns out, you can do this in a couple of ways.
You can implement this transformation by hand, or you can use the `zip(*lines)` trick:

In [37]:
lines = ["111", "000", "101", "101", "111"]
print(list(zip(*lines)))

[('1', '0', '1', '1', '1'), ('1', '0', '0', '0', '1'), ('1', '0', '1', '1', '1')]


This little trick would enable a solution like this:

In [38]:
from collections import Counter

with open(INPUT_FILE, "r") as f:
    lines = [line.strip() for line in f]
columns = zip(*lines)

gamma, epsilon = "", ""
for col in columns:
    # Count zeroes and ones in this column:
    counts = Counter(col)
    bit, _ = counts.most_common(1)[0]
    if bit == "0":
        gamma += "0"
        epsilon += "1"
    else:
        gamma += "1"
        epsilon += "0"

print(int(gamma, 2) * int(epsilon, 2))

749376


The `zip(*rows)` trick is quite convenient from the programmer's point of view: we write little code and get a really nice transformation for free.
However, this isn't really free: when we write `*lines` inside `zip`, it's as if we had written each and every single line inside `lines` as an argument to `zip`.

For the `lines = ["111", "000", "101", "101", "111"]` example above, it's as if we had written

In [39]:
list(zip("111", "000", "101", "101", "111"))

[('1', '0', '1', '1', '1'),
 ('1', '0', '0', '0', '1'),
 ('1', '0', '1', '1', '1')]

If our file becomes large, this might be something that Python can't handle.

However, there are other ways in which we can improve our counting, and also handle large files:

## Swapping iteration order

Imagine that you have a huge file.
You can't hold all of it in memory, so you can't just do what we have been doing.
So, if that's the case, how can we solve this?

Instead of going column by column, and then line by line, we can swap the order of traversal:
the outer loop can go over the lines, and the inner loop can go over the columns.

For that to work, we just need to initialise as many `Counter` objects as needed, and then traverse the line:

In [42]:
from collections import Counter

counters = []
with open(INPUT_FILE, "r") as f:
    for line in f:
        line = line.strip()
        if not counters:
            counters = [Counter() for _ in range(len(line))]
        for counter, bit in zip(counters, line):
            counter[bit] += 1

gamma, epsilon = "", ""
for counts in counters:
    bit, _ = counts.most_common(1)[0]
    if bit == "0":
        gamma += "0"
        epsilon += "1"
    else:
        gamma += "1"
        epsilon += "0"

print(int(gamma, 2) * int(epsilon, 2))

749376


However, this introduced an asymmetry that I am not a fan of:
the very first iteration has a special treatment, because that's when we initialise the list of empty counters.
Can't we do this in any other way..?

We can't know, beforehand, how many `Counter` objects we need, but we could go back to the `defaultdict` and use it to create default `Counter` objects when needed!
So, instead of having a list of `Counter` objects, we have a dictionary, and each key would be the corresponding column:

In [46]:
from collections import Counter, defaultdict

counters = defaultdict(Counter)
with open(INPUT_FILE, "r") as f:
    for line in f:
        line = line.strip()
        for col, bit in enumerate(line):
            counters[col][bit] += 1

gamma, epsilon = "", ""
for col in range(len(line)):
    bit, _ = counters[col].most_common(1)[0]
    if bit == "0":
        gamma += "0"
        epsilon += "1"
    else:
        gamma += "1"
        epsilon += "0"

print(int(gamma, 2) * int(epsilon, 2))

749376


Of course, at this point this is getting a bit too convoluted, no?
I was getting a bit carried away.
The idea is there, we are just doing it the wrong way.

## Counting once with column information

In fact, we can achieve the same effect with a single `Counter`.
In order to do that, we just have to do a small modification.
Instead of counting just the line, by itself, we pair up each bit with its column information.

Suppose this is a line:

In [47]:
line = "1100110"

If we feed it to a `Counter`, the `Counter` will count the zeroes and ones:

In [48]:
Counter(line)

Counter({'1': 4, '0': 3})

This ignores completely the fact that each bit represents a different column.
However, if we enumerate the line first, the `Counter` will understand that each bit comes from a different column:

In [49]:
Counter(enumerate(line))

Counter({(0, '1'): 1,
         (1, '1'): 1,
         (2, '0'): 1,
         (3, '0'): 1,
         (4, '1'): 1,
         (5, '1'): 1,
         (6, '0'): 1})

Now, if we have two different lines:

In [50]:
line1 = "1100110"
line2 = "1000111"

we can use `Counter(enumerate(...))` to count the bits, column-wise, and then use addition to combine the counts of each line:

In [53]:
counts = Counter(enumerate(line1)) + Counter(enumerate(line2))
counts

Counter({(0, '1'): 2,
         (1, '1'): 1,
         (2, '0'): 2,
         (3, '0'): 2,
         (4, '1'): 2,
         (5, '1'): 2,
         (6, '0'): 1,
         (1, '0'): 1,
         (6, '1'): 1})

Now, if you want to know how often `"1"` showed up in column `4`, you can just do:

In [56]:
counts[(4, "1")]

2

The correct result is `2` indeed because both `line1` and `line2` have a `"1"` in position `4`:

In [57]:
"1" == line1[4] == line2[4]

True

Thus, we can update our solution.

We just have to be careful with the way in which we build `gamma` and `epsilon` afterwards, because now our counting data has a different format:

In [4]:
from collections import Counter, defaultdict

counts = Counter()
with open(INPUT_FILE, "r") as f:
    for line in f:
        counts += Counter(enumerate(line.strip()))
print(counts)

gamma, epsilon = "", ""
for col in range(len(counts) // 2):
    zeroes, ones = counts[(col, "0")], counts[(col, "1")]
    if zeroes > ones:
        gamma += "0"
        epsilon += "1"
    else:
        gamma += "1"
        epsilon += "0"

print(int(gamma, 2) * int(epsilon, 2))

Counter({(4, '0'): 531, (3, '1'): 521, (10, '1'): 518, (5, '0'): 517, (9, '1'): 516, (0, '1'): 511, (2, '1'): 511, (8, '1'): 511, (1, '1'): 510, (11, '1'): 510, (7, '1'): 510, (6, '1'): 502, (6, '0'): 498, (7, '0'): 490, (11, '0'): 490, (1, '0'): 490, (2, '0'): 489, (0, '0'): 489, (8, '0'): 489, (9, '0'): 484, (5, '1'): 483, (10, '0'): 482, (3, '0'): 479, (4, '1'): 469})
749376


Notice how the printed `Counter` contains the information about all columns, and about the zeroes and ones.
In our `Counter` object, the counts from each column are all mixed together, but that is fine because we could access the specific results we wanted for each column.

We have looked quite closely at the way we count the bits, but we haven't thought a lot about the way we build the final `gamma` and `epsilon` values.

## Initialising gamma and epsilon

For me, personally, what makes me think about a better way to define `gamma` and `epsilon` is the fact that we want to do such a simple thing: add a zero or a one to the binary representation of each, and yet, we have to write so much code to do it.

And, to top it all off, there is such a nice symmetry here!
Couldn't we exploit it?

One thing we could do would be initialise `gamma` and `epsilon` to be filled with `"0"`s by default, and only assign the `"1"` when needed:

In [67]:
from collections import Counter, defaultdict

counts = Counter()
with open(INPUT_FILE, "r") as f:
    for line in f:
        counts += Counter(enumerate(line.strip()))

line_len = len(counts) // 2
gamma = ["0" for _ in range(line_len)]
epsilon = gamma[::]
for col in range(len(counts) // 2):
    zeroes, ones = counts[(col, "0")], counts[(col, "1")]
    if zeroes > ones:
        epsilon[col] = "1"
    else:
        gamma[col] = "1"

print(int("".join(gamma), 2) * int("".join(epsilon), 2))

749376


This might look like an improvement, but I am personally not a fan of it.
The initialisations look clunky.

## Bitwise operations

Another possibility would be to initialise `gamma` and `epsilon` to `0` and then manipulate the bits directly.
In case you didn't know, Python has a series of [operations to manipulate bits](https://docs.python.org/3/reference/expressions.html#binary-arithmetic-operations), and some of them can come in handy.

In particular, we care about the [left shifting operation](https://docs.python.org/3/reference/expressions.html#shifting-operations) and the [bitwise OR](https://en.wikipedia.org/wiki/Bitwise_operation).

The shifting operation takes two numbers and adds zeroes to the end of the binary expansion of the left argument:

In [1]:
bin(0b1 << 3)

'0b1000'

In [2]:
bin(0b101 << 3)

'0b101000'

The bitwise OR takes two numbers and compares them bit by bit, returning a new number with `1`s whenever there was at least a `1` on one side:

In [72]:
bin(0b1100 | 0b1010)

'0b1110'

With both combined, we can “turn on” a bit of a number.
For example, here is how you convert `0b00000` to `0b10000`:

In [73]:
bin(0b00000 | (1 << 4))

'0b10000'

Take your time to understand what just happened.

With this, we can build `gamma` and `epsilon` as integers:

In [81]:
from collections import Counter, defaultdict

counts = Counter()
with open(INPUT_FILE, "r") as f:
    for line in f:
        counts += Counter(enumerate(line.strip()))

gamma, epsilon = 0, 0
total_bits = len(counts) // 2
for col in range(total_bits):
    if counts[(col, "0")] > counts[(col, "1")]:
        epsilon |= 1 << (total_bits - col - 1)
    else:
        gamma |= 1 << (total_bits - col - 1)
        
print(gamma * epsilon)

749376


The left shift `<<` has a right argument of `total_bits - col - 1` because `col` is counting from `0`, but when `col` is small is when we want to shift a lot to the left, up to the leftmost column.

For example, suppose we have

In [78]:
total_bits = 5
col = 0

So, five columns and we are starting just now.

When `col` is `0`, we want `1 << ...` to produce the binary number `0b10000`, which has `4` zeroes to its right.
We get to the `4` by doing `total_bits - n`, and then `- 1` for a final adjustment:

In [80]:
bin(1 << (total_bits - col - 1))

'0b10000'

## Part 2 problem statement

(Adapted from [Advent of Code 2021, day 3](https://adventofcode.com/2021/day/3))

Next, you should find out the result of multiplying two new measures: the **oxygen generator rating** and the **CO2 scrubber rating**.

Both the oxygen generator rating and the CO2 scrubber rating are values that can be found in your input list - finding them is the tricky part.
Both values are located using a similar process that involves filtering out values until only one remains.
Before searching for either rating value, start with the full list of binary numbers from your input list and consider just the first bit of those numbers. Then:

 - keep only numbers selected by the bit criteria for the type of rating value for which you are searching. Discard numbers which do not match the bit criteria;
 - if you only have one number left, stop; this is the rating value for which you are searching;
 - otherwise, repeat the process, considering the next bit to the right.

The bit criteria depends on which type of rating value you want to find:

 - to find the **oxygen generator rating**, determine the most common value (`0` or `1`) in the current bit position, and keep only numbers with that bit in that position. If `0` and `1` are equally common, keep values with a `1` in the position being considered; and
 - to find the **CO2 scrubber rating**, determine the least common value (`0` or `1`) in the current bit position, and keep only numbers with that bit in that position. If `0` and `1` are equally common, keep values with a `0` in the position being considered.

For example, to determine the oxygen generator rating value using the same example input list from above:

 - Start with all 12 numbers and consider only the first bit of each number. There are more `1` bits (7) than `0` bits (5), so keep only the 7 numbers with a `1` in the first position: `11110`, `10110`, `10111`, `10101`, `11100`, `10000`, and `11001`.
 - Then, consider the second bit of the 7 remaining numbers: there are more `0` bits (4) than `1` bits (3), so keep only the 4 numbers with a `0` in the second position: `10110`, `10111`, `10101`, and `10000`.
 - In the third position, three of the four numbers have a `1`, so keep those three: `10110`, `10111`, and `10101`.
 - In the fourth position, two of the three numbers have a `1`, so keep those two: `10110` and `10111`.
 - In the fifth position, there are an equal number of `0` bits and `1` bits (one each). So, to find the oxygen generator rating, keep the number with a `1` in that position: `10111`.
 - As there is only one number left, stop; the oxygen generator rating is `10111`, or `23` in decimal.

Then, to determine the CO2 scrubber rating value from the same example above:

 - Start again with all 12 numbers and consider only the first bit of each number. There are fewer `0` bits (5) than `1` bits (7), so keep only the 5 numbers with a 0 in the first position: `00100`, `01111`, `00111`, `00010`, and `01010`.
 - Then, consider the second bit of the 5 remaining numbers: there are fewer `1` bits (2) than `0` bits (3), so keep only the 2 numbers with a `1` in the second position: `01111` and `01010`.
 - In the third position, there are an equal number of `0` bits and `1` bits (one each). So, to find the CO2 scrubber rating, keep the number with a `0` in that position: `01010`.
 - As there is only one number left, stop; the CO2 scrubber rating is `01010`, or `10` in decimal.

Finally, to find the life support rating, multiply the oxygen generator rating (`23`) by the CO2 scrubber rating (`10`) to get `230`.

**Use the binary numbers in your input list to calculate the oxygen generator rating and CO2 scrubber rating, then multiply them together**. What do you get? (Be sure to represent your answer in decimal, not binary.)

_Using the input file `03_binary_diagnostic.txt`, the result should be 2372923._

## Baseline solution

For this problem, we need to do a similar counting process to the one above, so we will use the techniques we have seen.

For our baseline solution, let's start by computing the oxygen generator rating:

In [19]:
from collections import Counter

with open(INPUT_FILE, "r") as f:
    lines = f.readlines()
bits = len(lines[0])
    
under_consideration = lines
for col in range(bits):
    if len(under_consideration) == 1:
        break
    
    # Count 0s and 1s in the appropriate column.
    counts = Counter(line[col] for line in under_consideration)
    pick = "0" if counts["0"] > counts["1"] else "1"
    under_consideration = [line for line in under_consideration if line[col] == pick]
oxygen_rating = under_consideration[0]

under_consideration = lines
for col in range(bits):
    if len(under_consideration) == 1:
        break
    
    counts = Counter(line[col] for line in under_consideration)
    pick = "0" if counts["0"] <= counts["1"] else "1"
    under_consideration = [line for line in under_consideration if line[col] == pick]
co2_rating = under_consideration[0]

print(int(oxygen_rating, 2) * int(co2_rating, 2))

2372923


This solution is already quite good:

 - we use a `Counter` to count the bits;
 - we use a [conditional expression](https://mathspp.com/blog/pydonts/conditional-expressions) to decide what is the current bit we should pick; and
 - we use a [list comprehension](https://mathspp.com/blog/pydonts/list-comprehensions-101) to keep only the lines that matter.

Now that we have some working code, we must look at it and figure out how we can improve it.

## DRY – Don't Repeat Yourself

One thing I like to do is look for parts of the code that are structurally the same, and see if I can merge them in some way.
In our code, we can see that computing the oxygen rating and the CO2 rating is the exact same thing; the only thing that changes is the way we pick the bit that matters:

```py
pick = "0" if counts["0"] > counts["1"] else "1"
# vs
pick = "0" if counts["0"] <= counts["1"] else "1"
```

And even then, these two lines are pretty similar.
They have a condition that depends on `counts["1"]` and `counts["0"]`, and that's what is used to determine whether we pick a `"1"` or a `"0"` next.

Therefore, we can write a function that does all this work.
As arguments, the function accepts the initial list of lines, and a predicate function (a function returning a Boolean value) that determines whether we should pick `"1"` or not.

For the oxygen rating, the predicate can look something like `lambda zeroes, ones: ones >= zeroes` and, for the CO2 rating, the predicate can look something like `lambda zeroes, ones: ones < zeroes`.
Notice that these predicates need to have the same arguments, and they take into account the fact that the oxygen rating picks a `"1"` in case of draw, while the CO2 rating picks a `"0"` in case of draw.

Here is a new version of the code:

In [20]:
from collections import Counter

with open(INPUT_FILE, "r") as f:
    lines = f.readlines()

def filter_by_count(lines, predicate):
    bits = len(lines[0])
    for col in range(bits):
        if len(lines) == 1:
            break
        
        counts = Counter(line[col] for line in lines)
        pick = "0" if predicate(counts["0"], counts["1"]) else "1"
        lines = [line for line in lines if line[col] == pick]

    return lines[0]
    
oxygen_rating = filter_by_count(lines, lambda zeroes, ones: zeroes > ones)
co2_rating = filter_by_count(lines, lambda zeroes, ones: zeroes <= ones)

print(int(oxygen_rating, 2) * int(co2_rating, 2))

2372923


Notice how the function body is _exactly the same_ as the two loops we had before, _except_ in the line that assigns `pick`.

## Operators in place of lambdas

Now I'd like to bring your attention to a small detail of the previous solution.
We had to implement two small `lambda` functions, but all they do is make use of the appropriate comparison operator: `>` for the oxygen rating and `<=` for the CO2 rating.

When everything we need from a `lambda` is to actually pass operands to a built-in operator, we should use the `operator` module.

For example, `operator.gt` is the function equivalent of `>`:

In [21]:
from operator import gt

In [22]:
gt(3, 0)  # 3 > 0

True

In [24]:
gt(3, 5)  # 3 > 5

False

Likewise, `le` is the `<=` operator, so we can make use of them in our code:

In [26]:
from collections import Counter
import operator

with open(INPUT_FILE, "r") as f:
    lines = f.readlines()

def filter_by_count(lines, predicate):
    bits = len(lines[0])
    for col in range(bits):
        if len(lines) == 1:
            break
        
        counts = Counter(line[col] for line in lines)
        pick = "0" if predicate(counts["0"], counts["1"]) else "1"
        lines = [line for line in lines if line[col] == pick]

    return lines[0]
    
oxygen_rating = filter_by_count(lines, operator.gt)
co2_rating = filter_by_count(lines, operator.le)

print(int(oxygen_rating, 2) * int(co2_rating, 2))

2372923


By using `operator`, we no longer have the need to define two `lambda` functions by ourselves, simplifying our code.

You can take a look at the [`operator` docs](https://docs.python.org/3/library/operator.html) to find out the names of the function equivalents of all the Python operators.

## Conditional loop

Our function `filter_by_count` exhibits the following pattern:

```py
for elem in iterable:
    if condition:
        break
```

This hints at the fact that maybe a `for` loop isn't the most appropriate loop here.
In fact, a `while` loop might be a good option: `while` loops are very suitable for when there is a condition that determines when to stop looping.
In our case, that condition is the fact that there is only one line left.

So, why did we use a `for` loop instead of a `while` loop?
Well, the fact is that the `for` loop is useful in order to keep track of the column we are in, but we could get rid of that if we introduced a `col` variable that we update by hand:

In [32]:
from collections import Counter
import operator

with open(INPUT_FILE, "r") as f:
    lines = f.readlines()

def filter_by_count(lines, predicate):
    col = 0
    while len(lines) > 1:
        counts = Counter(line[col] for line in lines)
        pick = "0" if predicate(counts["0"], counts["1"]) else "1"
        lines = [line for line in lines if line[col] == pick]
        col += 1

    return lines[0]
    
oxygen_rating = filter_by_count(lines, operator.gt)
co2_rating = filter_by_count(lines, operator.le)

print(int(oxygen_rating, 2) * int(co2_rating, 2))

2372923


However, we can do better than this!

As the iterations progress, what we want to do to the “following” column is completely independent from the “previous” column.
With that in mind, when we filter the lines for the next iteration, we can actually remove the current column.
Doing this means that the column we need to look at, next, is the very first column again.

If we do this, we no longer need `col`, and instead always index into the position `0` of the line.
However, now we need to keep track of the bit choices we have made in a `prefix` variable:

In [33]:
from collections import Counter
import operator

with open(INPUT_FILE, "r") as f:
    lines = f.readlines()

def filter_by_count(lines, predicate):
    prefix = ""
    while len(lines) > 1:
        counts = Counter(line[0] for line in lines)  # <- line[0] instead of line[col]
        pick = "0" if predicate(counts["0"], counts["1"]) else "1"
        prefix += pick
        lines = [line[1:] for line in lines if line[0] == pick]  # <- same

    return prefix + lines[0]  # re-add the prefix to the string we found
    
oxygen_rating = filter_by_count(lines, operator.gt)
co2_rating = filter_by_count(lines, operator.le)

print(int(oxygen_rating, 2) * int(co2_rating, 2))

2372923


## Recursive solution

The two solutions above are _very_ similar, but one can argue that the second one (where we keep dropping the first column) is more natural for recursion.

Let's turn the two solutions into recursive functions.

To do that, the condition on the `while` loop becomes the base case for the recursive function.
Then, all other statements remain the same, except we need to call the function recursively instead of updating the `prefix` or the `col` variables:

In [39]:
def filter_by_count_with_col(lines, predicate, col=0):
    if len(lines) == 1:
        return lines[0]
    
    counts = Counter(line[col] for line in lines)
    pick = "0" if predicate(counts["0"], counts["1"]) else "1"
    lines = [line for line in lines if line[col] == pick]
    return filter_by_count_with_col(lines, predicate, col + 1)
        

def filter_by_count_with_prefix(lines, predicate):
    if len(lines) == 1:
        return lines[0]

    counts = Counter(line[0] for line in lines)
    pick = "0" if predicate(counts["0"], counts["1"]) else "1"
    lines = [line[1:] for line in lines if line[0] == pick]
    return pick + filter_by_count_with_prefix(lines, predicate)

The one you pick is mostly a matter of personal preference; I tend to prefer the latter because there is one less argument to worry about.

Recursion can be quite an elegant construct, from the mathematical point of view, but there are some drawbacks to using it in Python (unfortunately).

One of them is related to memory costs.
In fact, when we call the function recursively, all the memory that we are using at the current point in the function is kept around.
Thus, as we go deep into the multiple levels of recursion, each single level is using up some memory to keep track of the variable `lines`, for example.
This can be quite bad if the input list is very large.

If you are curious about it, you can read a bit more about recursion in Python, and some of its unfortunate drawbacks, [here](https://mathspp.com/blog/pydonts/watch-out-for-recursion).

## Space-time complexity analysis

Feel free to skip this if you couldn't care less.

In order to analyse the space-time complexity of our solutions, we first need to identify the dimensions of the problem that matter.
In our case, those are the number of bit patterns we have (the number of lines) that we will call $n$, and the number of columns, that we will call $k$.

Up until now, most of our solutions (to both tasks) had a space complexity of $O(nk)$, because we stored _everything_ in memory.

As for time complexity, this also isn't very complicated to estimate.
Both the imperative and the recursive solutions have a main body that is $O(n)$ in time complexity, because it goes over the list of lines once.
Then, the main body runs a couple of times, but it runs _at most_ once per column, so the total time complexity is also $O(nk)$.

For the case of the CO2 rating, we can try to say something more, because at every step we reduce the number of lines being considered by half at least.
Say you have a quantity $m$ that keeps getting reduced into half.
This halving process will need $\log_2 m$ steps to reduce $m$ to $1$.
In a similar way, we can expect the CO2 rating to go from $n$ lines to $1$ line in $O(\log_2 n)$ time.
However, for our problem, we know that we will always get a single line after applying the filtering process over all the columns, even if $\log_2 n$ is much larger than $k$.
Thus, it might be more accurate to say that the time complexity is $O(nk)$ instead of saying $O(n\log_2 n)$.

Now, a question arises: can we solve this task in a more space-efficient manner?

The answer is: _absolutely_.

The content that follows is a bit more involved; feel free to skip it if you don't want to get into a deep discussion about space efficienty.

## Space-efficient solution

A space-efficient solution will revolve around a very important fact:
we don't need to keep all the strings around.
We only need to keep the counts of prefixes, and we can do that with a tree-like representation like the following:

![](03_binary_trie.png)

Looking at the figure above, the tree we see represents the counts of a total of

In [40]:
0 + 3 + 1 + 8 + 1 + 6 + 5 + 4

28

bit patterns.

Imagine we are just starting out, and we need to determine how many zeroes and ones there are in the first column.
We can figure out how many patterns start with a zero by looking at the top half of the tree ($0 + 3 + 1 + 8$) and we can figure out how many patterns start with a one by looking at the bottom half of the tree.

Thus, if we build this structure when we first read the file, we get to save a lot of space:
we only need to store one integer per each possible bit pattern, and there are $2^k$ bit patterns.
(We are assuming that the number of lines $n$ is much larger than the number of possible bit patterns, $2^k$.)

So, how can we represent this structure?

We don't need to get very fancy, a dictionary with keys `"0"` and `"1"` will suffice!
Let's call it a `tree_counter`.
For example, here is the representation of the figure above:

In [2]:
tree_counter = {
    "0": {
        "0": {
            "0": 0,
            "1": 3,
        },
        "1": {
            "0": 1,
            "1": 8,
        },
    },
    "1": {
        "0": {
            "0": 1,
            "1": 6,
        },
        "1": {
            "0": 5,
            "1": 4,
        },
    },
}

We just need some helper functions to deal with these structures:

 - a function that initialises a dictionary like this, with `k` levels of depth and all counts as `0`;
 - a function that takes a string and increases the count of the correct position; and
 - a function to sum all the counts in the given dictionary.

Here is how we can initialise the empty structure:

In [3]:
def init_tree_dict(levels):
    if not levels:
        return 0

    return {
        "0": init_tree_dict(levels - 1),
        "1": init_tree_dict(levels - 1),
    }

In [4]:
init_tree_dict(3)

{'0': {'0': {'0': 0, '1': 0}, '1': {'0': 0, '1': 0}},
 '1': {'0': {'0': 0, '1': 0}, '1': {'0': 0, '1': 0}}}

Now, we can define another recursive function that uses the argument string to navigate through the structure and increase the count associated with that string:

In [6]:
def add_count(tree_dict, string):
    if len(string) == 1:
        tree_dict[string] += 1
    else:
        add_count(tree_dict[string[0]], string[1:])

In [8]:
print(tree_counter["1"]["1"]["1"])  # How many times have we seen the string "111"?
add_count(tree_counter, "111")      # Oh, hey, we just saw the string "111"!
print(tree_counter["1"]["1"]["1"])  # Now, how many times have we seen the string "111"?

5
6


Finally, we write a function that goeso down the structure and adds up all counts:

In [9]:
def add_up(tree_dict):
    if isinstance(tree_dict, dict):
        return sum(add_up(sub_dict) for sub_dict in tree_dict.values())
    else:
        return tree_dict

print(add_up(tree_counter))  # It's 29 because we added a count just now.

30


With this in place, we can write a nice space-efficient solution:

In [17]:
def init_tree_dict(levels):
    if not levels:
        return 0

    return {
        "0": init_tree_dict(levels - 1),
        "1": init_tree_dict(levels - 1),
    }

def add_count(tree_dict, string):
    if len(string) == 1:
        tree_dict[string] += 1
    else:
        add_count(tree_dict[string[0]], string[1:])

def add_up(tree_dict):
    if isinstance(tree_dict, dict):
        return sum(add_up(sub_dict) for sub_dict in tree_dict.values())
    else:
        return tree_dict

# Parse the file and fill the tree_counter.
with open(INPUT_FILE, "r") as f:
    # Take the first line to figure out the depth we need; assumes the file isn't empty.
    first_line = next(f)
    tree_counter = init_tree_dict(len(first_line.strip()))
    f.seek(0)
    for line in f:
        add_count(tree_counter, line.strip())

print(add_up(tree_counter))  # This tells us how many lines the file had.

1000


This parses the file and builds up the count.

**Challenge**:
if you inspect `tree_counter`, you will see many positions are empty.
They were initialised to be `0`, but they remained empty.
Can you change the way `tree_counter` is defined, so that it only includes the positions that matter?

Now, we just need to modify the `filter_by_count` function to accept something like the `tree_counter` instead of all of the lines of the file.
Then, we keep going through the tree structure, until we reach the end of the dictionary:

In [22]:
def filter_by_count(tree_counter, predicate):
    prefix = ""
    while isinstance(tree_counter, dict):
        zeroes, ones = add_up(tree_counter["0"]), add_up(tree_counter["1"])
        pick = "0" if predicate(zeroes, ones) else "1"
        prefix += pick
        tree_counter = tree_counter[pick]
    return prefix
    
oxygen_rating = filter_by_count(tree_counter, operator.gt)
co2_rating = filter_by_count(tree_counter, operator.le)

print(int(oxygen_rating, 2) * int(co2_rating, 2))

2353568


But this answer is wrong!

Why?
Because the `co2_rating` variable is being computed in the wrong way!
The problem statement tells us to consider the patterns available, but our new function is also taking into account patterns that don't exist, and if a pattern doesn't exist, it's count is always 0!

Therefore, when we assign to `pick`, we need to make sure that we only assign `"0"` to `pick` if the count of zeroes is strictly greater than 0.
Similarly, we _need_ to assign `"0"` to `pick` if the count of ones is 0.

To better understand what I mean, take a look at the figure below:

![](03_binary_trie_co2.png)

If we are computing the variable `co2_rating`, then we always choose the path with smallest number.
When we do that, we end up picking the path to the pattern `"000"`, even though that pattern doesn't exist!
Therefore, to prevent those things from happening, we have to add a couple of additional checks:

 - if the predicate function returns `True`, we assign `"0"` to `pick`, **but** we must also make sure that the variable `zeroes` is greater than 0, to prevent picking a path with no patterns;
 - similarly, we **have** to assign `"0"` to `pick` if `ones == 0`, because that means there is no pattern down that road.

Here is the modified conditional expression:

In [32]:
def filter_by_count(tree_counter, predicate):
    prefix = ""
    while isinstance(tree_counter, dict):
        zeroes, ones = add_up(tree_counter["0"]), add_up(tree_counter["1"])
        # Ensure we go down a path with existing patterns:
        pick = "0" if predicate(zeroes, ones) and zeroes > 0 or ones == 0 else "1"
        prefix += pick
        tree_counter = tree_counter[pick]
    return prefix
    
oxygen_rating = filter_by_count(tree_counter, operator.gt)
co2_rating = filter_by_count(tree_counter, operator.le)

print(int(oxygen_rating, 2) * int(co2_rating, 2))

2372923


This representation of the patterns is smaller because the many lines of input don't need to be represented in full: instead, the patterns that have a similar prefix can share that information in the structure.

This representation was heavily influenced by a data structure that is called a [_trie_](https://en.wikipedia.org/wiki/Trie), sometimes also referred to as a _prefix tree_.

In terms of space, holding everything in memory was $O(nk)$.
With this structure, we have something that is $O(2^k)$, which is supposed to match $O(n)$ roughly.
However, we have no guarantee that this is the case.
In order to guarantee that our space representation is actually $O(n)$, we have to solve the challenge from above.

## Linear space

If we want to make sure we use only the exact space we need, we can use `defaultdict` again to populate the dictionary automatically for us.
This means we have to refactor our helper functions slightly:

In [55]:
from collections import defaultdict

def new_tree_dict():
    return defaultdict(new_tree_dict)    # <- Recursive defaultdict structure;
                                         # this builds the dictionary as needed.

def add_count(tree_dict, string):
    if len(string) == 1:
        # Ensure we can increment the count by setting the default value to 0.
        tree_dict.setdefault(string, 0)
        tree_dict[string] += 1
    else:
        add_count(tree_dict[string[0]], string[1:])

def add_up(tree_dict):
    if isinstance(tree_dict, dict):
        return sum(add_up(sub_dict) for sub_dict in tree_dict.values())
    else:
        return tree_dict

tree_counter = new_tree_dict()
with open(INPUT_FILE, "r") as f:
    # No longer need to peek at the first line of f.
    for line in f:
        add_count(tree_counter, line.strip())

After we do this, we fix the function `filter_by_count` and we are good to go.

The “fix” is simple: we just need to use the `.get` function instead of accessing the `tree_counter` directly.
Why do we do this?
We do this so that the `tree_counter` doesn't grow if we try to access a path that isn't holding any patterns.

In [57]:
def filter_by_count(tree_counter, predicate):
    prefix = ""
    while isinstance(tree_counter, dict):
        zeroes, ones = add_up(tree_counter.get("0", 0)), add_up(tree_counter.get("1", 0))
        pick = "0" if predicate(zeroes, ones) and zeroes > 0 or ones == 0 else "1"
        prefix += pick
        tree_counter = tree_counter[pick]
    return prefix
    
oxygen_rating = filter_by_count(tree_counter, operator.gt)
co2_rating = filter_by_count(tree_counter, operator.le)

print(int(oxygen_rating, 2) * int(co2_rating, 2))

2372923


This solution is now $O(n)$ in terms of space, because the dictionary only has entries for the patterns that actually exist.

It takes up less space than holding all the patterns because the patterns are sharing all their prefixes.

## Conclusion

Even simple-looking problems can reveal much more depth to it, if you try to come up with increasingly efficient solutiosn.
Of course, in the real world you have to find a balance between how much time you spend coding and the performance benefits you get.

For example, in this particular case, I spent some hours thinking about these space-efficient solutions, but I gained very little: my computer could crack the problem with the less efficient solution in _literally_ the blink of an eye and it could hold everything in memory, so I'm not really gaining anything by implementing a more efficient solution.

If you have any questions, suggestions, remarks, recommendations, corrections, or anything else, you can reach out to me [on Twitter](https://twitter.com/mathsppblog) or via email to rodrigo at mathspp dot com.